# Westeros - Introducing emission taxes

In the previous tutorial (``westeros_baseline_gas.ipynb``), we introduced a stylized energy systems model including gas plants. 

Now, we will tackle the complementary policy to emissions constraints, namely emissions taxes.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario with gas plant (``westeros_baseline_gas.ipynb``) and solved it successfully

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform()

In [ ]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='baseline_gas')

## Setting an emissions tax

We visualize the effects of emissions taxes on the energy mix in Westeros.

In [ ]:
scen_tax = base.clone(model, 'carbon_tax_gas','introducing a carbon tax',
                      keep_solution=False)
scen_tax.check_out()

In [ ]:
scen_tax.add_set('emission', 'CO2')
scen_tax.add_cat('emission', 'GHG', 'CO2')

According to [IEA 2019](https://www.iea.org/reports/the-role-of-gas-in-todays-energy-transitions), "coal-to-gas switching reduces emissions by 50% when producing electricity", therefore, the emissions factor for gas is set to half that of coal.

In [ ]:
year_df = scen_tax.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen_tax.set('year')
country = 'Westeros'

scen_tax.add_set('emission', 'CO2')
scen_tax.add_cat('emission', 'GHG', 'CO2')

# we now add CO2 emissions to the coal and gas powerplants
base_emission_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'unit': 'tCO2/kWa',
}

# adding new units to the model library (needed only once)
mp.add_unit('tCO2/kWa')
mp.add_unit('MtCO2')

emission_factor = {
    'coal_ppl': 7.4,
    'gas_ppl': 3.7,
}

for tec, val in emission_factor.items():
    df = make_df(base_emission_factor, technology=tec, emission='CO2', value=val)
    scen_tax.add_par('emission_factor', df)

Setting an initial base CO2 tax at 50 USD / tCO2. This initial tax will be later modified to check the effect of varying it on the energy mix.

In [ ]:
base_tCO2_tax = 50
scen_tax.add_set('type_year', [700, 710, 720])

base_tax_emission = {
    'node': 'Westeros',
    'type_year': [700, 710, 720],
    'type_tec': 'all',
    'unit': 'USD/tCO2',
    'type_emission': 'GHG',
    'value': [base_tCO2_tax, base_tCO2_tax, base_tCO2_tax],
}

# in case you need to add the new unit to the platform
mp.add_unit('USD/tCO2')

tax_emission = make_df(base_tax_emission)
scen_tax.add_par('tax_emission', tax_emission)
scen_tax.par('tax_emission')

In [ ]:
scen_tax.commit(comment='setting taxes on emissions to base_tCO2_tax')
scen_tax.set_as_default()

In [ ]:
scen_tax.solve()

## Plotting Results

We make use of some custom code for plotting the results; see `tools.py` in the tutorial directory.

In [ ]:
from tools import Plots
p = Plots(scen_tax, country, firstyear=model_horizon[0])

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'gas_ppl','wind_ppl'])

### Capacity

How much capacity of each plant is installed in each period?

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'gas_ppl','wind_ppl'])

### Electricity Price

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

## Plotting the Activity when emission taxes are varied

In [ ]:
for tCO2_tax in range(25, 70, 5):
    #Creating temporary scenario to modify tCO2_tax
    temp_scen_tax = scen_tax.clone(model, 'carbon_tax_temp','temporary scenario',
                      keep_solution=False)
    temp_scen_tax.check_out()
    
    #Setting emission tax level
    temp_tax_emission = {
        'node': 'Westeros',
        'type_year': [700, 710, 720],
        'type_tec': 'all',
        'unit': 'USD/tCO2',
        'type_emission': 'GHG',
        'value': [tCO2_tax, tCO2_tax, tCO2_tax],
    }
    tax_emission = make_df(temp_tax_emission)
    temp_scen_tax.add_par('tax_emission', tax_emission)
    
    #Solving temporary scenario
    temp_scen_tax.commit(comment='temporary scenario')
    temp_scen_tax.solve()
    
    #Plotting Activity
    p = Plots(temp_scen_tax, country, firstyear=model_horizon[0])
    print(tCO2_tax)
    p.plot_activity(baseyear=True, subset=['coal_ppl', 'gas_ppl','wind_ppl'])

## Close the connection to the database

In [ ]:
mp.close_db()